In [1]:
import pandas as pd
import numpy as np
import glob

In [2]:
metadata_df = pd.read_csv("~/mrc/project/rna-seq/data/sita_eisa_inflammation_metadata.txt", sep="\t")

study = 'Inflammation'
notes = 'LPSinWTonly'

metadata_df = metadata_df[metadata_df['Study'] == study]

metadata_df.head()

# General GSE stats
total_gse = len(metadata_df['GSE'].unique())
total_comp = len(metadata_df.loc[metadata_df['SITA'] != ''])

print(f"""
Total GSEs: {total_gse}
Total comparisons: {total_comp}
""")

,GSE,Study,Organism,Cell/Tissue type,Treatment(s),Time (hr),Concentration (µg/ml),Genotype(s),Nmin,Nmax,...,lps_only,p_intron,EISA_nsig,EISA_Up:Down,DESeq_nsig,DESeq_Up:Down,SITA,Remarks,Other seq,Other comments
2,GSE103719,Inflammation,Homo sapiens,HEK293T cells,DMSO vs. THAP,2,2,None,4.0,4.0,...,0.0,0.05265,485.0,0.061269,971.0,1.568783,TRUE,Positive control with Barbara's result (human).,"TED-seq, Ribo-seq, PRO-seq",NaN
3,GSE105155,Inflammation,Mus musculus,BV2 microglia,Control vs. LPS,3,1,TET2 (siRNA KD) vs. WT,2.0,3.0,...,1.0,0.15230,307.0,0.358407,2104.0,1.446512,TRUE?,NaN,None,NaN
4,GSE105155,Inflammation,Mus musculus,BV2 microglia,Control vs. LPS,3,1,TET2 (siRNA KD) vs. WT,2.0,3.0,...,1.0,0.13380,108.0,0.687500,1660.0,1.331461,FALSE,NaN,None,NaN
5,GSE106789,Inflammation,Mus musculus,Granulocyte-monocyte progenitor cells,PBS vs. LPS injection (intravenous),6,NaN,Nlrp3 (-/-) Ldlr (-/-),3.0,4.0,...,1.0,0.10880,158.0,0.362069,1814.0,1.402649,TRUE?,No WT control.,None,NaN
6,GSE108770,Inflammation,Mus musculus,N9 microglia,Normal vs. LPS,3,0.1,None,3.0,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Normal is SINGLE, LPS is PAIRED - cannot proce...",ChIP-seq,NaN


In [4]:
organism = 'Mus musculus'
cell_type = 'macrophage'

if cell_type == 'macrophage':
    cell_type_select = ['BMDM', 'macrophage', 'Macrophage']
elif cell_type == 'microglia':
    cell_type_select = ['microglia', 'Microglia']

select_df = metadata_df.loc[metadata_df['Organism'] == organism]
select_df = select_df[select_df['Cell/Tissue type'].str.contains('|'.join(cell_type_select))]

lps_wt_df = select_df.loc[(metadata_df['wt_comp'] == 1) &
                          (metadata_df['lps_only'] == 1)]

gses = lps_wt_df['GSE'].unique()

print(f"No. of GSEs: {len(gses)}")
print(gses)

lps_wt_df

No. of GSEs: 16
['GSE110243' 'GSE119380' 'GSE120807' 'GSE122292' 'GSE123596' 'GSE124410'
 'GSE125036' 'GSE131561' 'GSE134456' 'GSE147943' 'GSE157786' 'GSE55385'
 'GSE82043' 'GSE86922' 'GSE92618' 'GSE98563']


,GSE,Study,Organism,Cell/Tissue type,Treatment(s),Time (hr),Concentration (µg/ml),Genotype(s),Nmin,Nmax,...,lps_only,p_intron,EISA_nsig,EISA_Up:Down,DESeq_nsig,DESeq_Up:Down,SITA,Remarks,Other seq,Other comments
8,GSE110243,Inflammation,Mus musculus,BMDM (treated),"Untreated vs. LPS, +/-Dex",1,0.01,NELF-B (KO) vs. WT,2.0,3.0,...,1.0,0.017210,18.0,17.000000,220.0,4.116279,FALSE*?,NaN,None,NaN
19,GSE119380,Inflammation,Mus musculus,BMDM (treated),Untreated vs. LPS,6,NaN,SRF (KO) vs. WT,3.0,3.0,...,1.0,0.046430,1780.0,0.142490,7027.0,1.043327,TRUE,NaN,None,NaN
24,GSE120807,Inflammation,Mus musculus,BMDM (treated),"Unstimulated vs. LPS, +/- IFNa, +/- IFNg","4,8/4,8/4",1,None,3.0,9.0,...,1.0,0.014907,605.0,0.559278,4344.0,1.016713,TRUE?,NaN,None,NaN
26,GSE122292,Inflammation,Mus musculus,BMDM (treated),Untreated vs. LPS,"0.5, 1",0.01,NELF-B (KO) vs. WT,3.0,3.0,...,1.0,0.018010,14.0,NaN,22.0,NaN,FALSE*?,Global upregulation.,None,NaN
27,GSE122292,Inflammation,Mus musculus,BMDM (treated),Untreated vs. LPS,"0.5, 1",0.01,NELF-B (KO) vs. WT,3.0,3.0,...,1.0,0.017080,202.0,1.126316,268.0,2.621622,FALSE,NaN,None,NaN
30,GSE123596,Inflammation,Mus musculus,BMDM (treated),Normal vs. LPS,"1,18",0.1,None,3.0,3.0,...,1.0,0.022030,895.0,0.110422,1488.0,0.957895,TRUE,Positive control with Barbara's result (mouse).,None,NaN
32,GSE124410,Inflammation,Mus musculus,BMDM (treated),Untreated vs. LPS,"6, 16, 32",0.1,STK40 (KO) vs. WT,1.0,2.0,...,1.0,0.035620,0.0,NaN,3401.0,1.215635,NaN,NaN,None,NaN
33,GSE124410,Inflammation,Mus musculus,BMDM (treated),Untreated vs. LPS,"6, 16, 32",0.1,STK40 (KO) vs. WT,1.0,2.0,...,1.0,0.038830,163.0,0.442478,6407.0,1.134955,TRUE?,NaN,None,redo o
35,GSE124410,Inflammation,Mus musculus,BMDM (treated),Untreated vs. LPS,"6, 16, 32",0.1,STK40 (KO) vs. WT,1.0,2.0,...,1.0,0.039790,2733.0,1.735736,5910.0,1.065711,FALSE,NaN,None,NaN
37,GSE125036,Inflammation,Mus musculus,BMDM (treated),"Normal vs. LPS, +/- serine deprivation",4,0.1,None,2.0,2.0,...,1.0,0.019780,158.0,0.423423,3138.0,1.087824,TRUE?,NaN,None,NaN


In [6]:
# Create proteostasis table
proteo_list = pd.read_csv("/Users/pomato/mrc/project/rna-seq/data/Proteolist.csv", sep="\t")

if organism == 'Homo sapiens':
    proteo_df = pd.DataFrame({'gene_id': proteo_list['Human_gene_ID'],
                               'symbol': proteo_list['Human_Symbol']})
elif organism == 'Mus musculus':
    proteo_df = pd.DataFrame({'gene_id': proteo_list['Mouse_gene_ID'],
                               'symbol': proteo_list['Mouse_Symbol']})

proteo_df.dropna(subset=['gene_id'], inplace=True)

print(f"No. of proteostasis genes: {len(proteo_df)}")

for gse in gses:
    proteo_df[gse] = np.nan

for gse in gses:
    # Load DESeq2 output table
    deseq_files = glob.glob(f"/Users/pomato/mrc/project/rna-seq/data/deseq_outputs/{gse}*.txt")

    if len(deseq_files) > 0:
        deseq_df = pd.read_csv(deseq_files[0], sep="\t")

        print(f"Filtering for proteostasis genes in {gse} DESeq2 output table...")
        deseq_df_filt = deseq_df[deseq_df.index.isin(proteo_df['gene_id'])]
        print(f"{len(deseq_df_filt)} out of {len(deseq_df)} genes ({len(deseq_df_filt) / len(deseq_df) * 100}%) kept.")

        fc_df = proteo_df
        padj_df = proteo_df
        
        print("Filling proteostasis table with fold-change and padj values of corresponding genes...")
        for i in deseq_df_filt.index.values:
            fc_df.loc[fc_df.gene_id == i, gse] = deseq_df_filt.loc[i, 'log2FoldChange']
            padj_df.loc[padj_df.gene_id == i, gse] =deseq_df_filt.loc[i, 'padj']

    else:
        print(f"DESeq2 output table not found for {gse}. Skipping...")

# Fill table for fold-change and padj


# heatmap_df.to_csv(f"~/mrc/project/sita_eisa/data/{study}_{notes}_{organism}_{cell_type}_GSEs.txt", sep="\t", header=True, index=False)



No. of proteostasis genes: 2034
DESeq2 output table not found for GSE110243. Skipping...
DESeq2 output table not found for GSE119380. Skipping...
DESeq2 output table not found for GSE120807. Skipping...
DESeq2 output table not found for GSE122292. Skipping...
DESeq2 output table not found for GSE123596. Skipping...
DESeq2 output table not found for GSE124410. Skipping...
Filtering for proteostasis genes in GSE125036 DESeq2 output table...
2021 out of 55401 genes (3.6479485929856863%) kept.
Filling proteostasis table with fold-change and padj values of corresponding genes...
DESeq2 output table not found for GSE131561. Skipping...
DESeq2 output table not found for GSE134456. Skipping...
DESeq2 output table not found for GSE147943. Skipping...
DESeq2 output table not found for GSE157786. Skipping...
DESeq2 output table not found for GSE55385. Skipping...
DESeq2 output table not found for GSE82043. Skipping...
DESeq2 output table not found for GSE86922. Skipping...
DESeq2 output table not

In [7]:
fc_df

,gene_id,symbol,GSE110243,GSE119380,GSE120807,GSE122292,GSE123596,GSE124410,GSE125036,GSE131561,GSE134456,GSE147943,GSE157786,GSE55385,GSE82043,GSE86922,GSE92618,GSE98563
1,ENSMUSG00000015671,Psma2,NaN,NaN,NaN,NaN,NaN,NaN,4.418526e-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ENSMUSG00000029778,Adcyap1r1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ENSMUSG00000042541,Sem1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,ENSMUSG00000029368,Alb,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,ENSMUSG00000026024,Als2,NaN,NaN,NaN,NaN,NaN,NaN,1.682019e-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2031,ENSMUSG00000034636,Zyg11b,NaN,NaN,NaN,NaN,NaN,NaN,2.621501e-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2032,ENSMUSG00000024317,Rnf138,NaN,NaN,NaN,NaN,NaN,NaN,8.166079e-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2033,ENSMUSG00000060441,Trim5,NaN,NaN,NaN,NaN,NaN,NaN,8.009260e-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11000,ENSMUSG00000078622,Ccdc47,NaN,NaN,NaN,NaN,NaN,NaN,4.290772e-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
